<a href="https://colab.research.google.com/github/KhandkerHaque/titanic/blob/main/notebooks/classification_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a classifier with Titanic Data Set

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display
%matplotlib inline

sns.set(style="whitegrid", font_scale=1.75)

# Titanic dataset:

On April 15, 1912, the largest passenger liner ever made collided with an iceberg during her maiden voyage. When the Titanic sank it killed 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships. One of the reasons that the shipwreck resulted in such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others.

The titanic.csv file contains data for 887 of the real Titanic passengers. Each row represents one person. The columns describe different attributes about the person including whether they survived (S
), their age (A
), their passenger-class (C
), their sex (G
) and the fare they paid (X
).

In [5]:
#Load the dataset using Pandas
# train and test
data_path = 'ML_Titanic'
train = pd.read_csv('/content/gdrive/My Drive/'+data_path+'/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/'+data_path+'/test.csv')

print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [4]:
import os

# Replace 'your_directory_path' with the path to the directory you want to explore
directory_path = '/content/gdrive/My Drive/'

for root, dirs, files in os.walk(directory_path):
    print(f"Directory: {root}")
    for name in files:
        print(f"  File: {name}")

Directory: /content/gdrive/My Drive/
  File: Resume_CL_Per Stmnt _fs_ Khandker A Haque_LHMC.docx
  File: Global Health Term Paper .gdoc
  File: Cover letter_KAH_ Morgan Stanley NYPB.docx
  File: LSSGB CAPM MLM AUPHA ACHE UIPath (1).pdf
  File: LSSGB CAPM MLM AUPHA ACHE UIPath.pdf
  File: MBA MHA MPH MBBS (1).pdf
  File: MBA MHA MPH MBBS.pdf
  File: MHA MBA complete Transcript unofficial (1).pdf
  File: MHA MBA complete Transcript unofficial.pdf
  File: WES MPH MBBS trancript evaluated (1).pdf
  File: WES MPH MBBS trancript evaluated.pdf
  File: Dr S old Rec Letter.pdf
  File: Recommendation Letter _Dr Steven Szydlowski_DBA scholarship.pdf
  File: Scholarship Essay _ Khandker A Haque (1).docx
  File: Scholarship Essay _ Khandker A Haque.docx
  File: WIN_20221125_11_53_53_Pro.mp4
  File: DBA Admission Offer letter.pdf
  File: 1st Payment Proof to Westcliff University _ 11 28 2022.pdf
  File: this_message_in_html.htm
  File: Untitled document (4).gdoc
  File: Sessions MAY-JUNE-JULY_May 24

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
# create new features
# for Name
train['Name_len']=train.Name.str.len()

In [9]:
train['Ticket_First']=train.Ticket.str[0]

In [10]:
train['FamilyCount']=train.SibSp+train.Parch

In [11]:
train['Cabin_First']=train.Cabin.str[0]

In [12]:
# Regular expression to get the title of the Name
train['title'] = train.Name.str.extract('\, ([A-Z][^ ]*\.)',expand=False)

<>:2: SyntaxWarning: invalid escape sequence '\,'
<>:2: SyntaxWarning: invalid escape sequence '\,'
/tmp/ipython-input-2793989880.py:2: SyntaxWarning: invalid escape sequence '\,'
  train['title'] = train.Name.str.extract('\, ([A-Z][^ ]*\.)',expand=False)


In [13]:
train.title.value_counts().reset_index()

,title,count
0,Mr.,517
1,Miss.,182
2,Mrs.,125
3,Master.,40
4,Dr.,7
5,Rev.,6
6,Mlle.,2
7,Major.,2
8,Col.,2
9,Don.,1


**Missing Value treatment**

Having missing values in a dataset can cause errors with some machine learning algorithms and either the rows that has missing values should be removed or imputed
Imputing refers to using a model to replace missing values.

There are many options we could consider when replacing a missing value, for example:

constant value that has meaning within the domain, such as 0, distinct from all other values.
value from another randomly selected record.
mean, median or mode value for the column.
value estimated by another predictive model.


In [14]:
# impute the missing Fare values with the mean Fare value
train.Fare.fillna(train.Fare.mean(),inplace=True)

/tmp/ipython-input-339630426.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train.Fare.fillna(train.Fare.mean(),inplace=True)


In [15]:
# impute the missing Age values with the mean Fare value
train.Age.fillna(train.Age.mean(),inplace=True)

/tmp/ipython-input-2908588107.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train.Age.fillna(train.Age.mean(),inplace=True)


sometimes it is more reasonabale to drop the columns

In [ ]:
# We see that a majority 77% of the Cabin variable has missing values.
# Hence will drop the column from training a machine learnign algorithem
train.Cabin.isnull().mean()

0.7710437710437711

In [16]:
trainML = train[['Survived', 'Pclass', 'Sex', 'Age', 'Parch',
       'Fare', 'Embarked', 'Name_len', 'Ticket_First', 'FamilyCount',
       'title']]

In [17]:
trainML

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Name_len,Ticket_First,FamilyCount,title
0,0,3,male,22.000000,0,7.2500,S,23,A,1,Mr.
1,1,1,female,38.000000,0,71.2833,C,51,P,1,Mrs.
2,1,3,female,26.000000,0,7.9250,S,22,S,0,Miss.
3,1,1,female,35.000000,0,53.1000,S,44,1,1,Mrs.
4,0,3,male,35.000000,0,8.0500,S,24,3,0,Mr.
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,13.0000,S,21,2,0,Rev.
887,1,1,female,19.000000,0,30.0000,S,28,1,0,Miss.
888,0,3,female,29.699118,2,23.4500,S,40,W,3,Miss.
889,1,1,male,26.000000,0,30.0000,C,21,1,0,Mr.


In [18]:
# drop rows of missing values
trainML = trainML.dropna()

In [19]:
# check the datafram has any missing values
trainML.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,0
Parch,0
Fare,0
Embarked,0
Name_len,0
Ticket_First,0
FamilyCount,0


**Encoding categorical variables**

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(trainML['Sex'].unique())
le.classes_

array(['female', 'male'], dtype=object)

In [21]:
trainML['Sex']=le.transform(trainML['Sex'])

/tmp/ipython-input-3222977609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainML['Sex']=le.transform(trainML['Sex'])


In [22]:
le.fit(trainML['title'].unique())
trainML['title'] = le.transform(trainML['title'])

/tmp/ipython-input-4059273603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainML['title'] = le.transform(trainML['title'])


In [23]:
le.fit(trainML['Ticket_First'].unique())
trainML['Ticket_First'] = le.transform(trainML['Ticket_First'])

/tmp/ipython-input-237107148.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainML['Ticket_First'] = le.transform(trainML['Ticket_First'])


In [24]:
le.fit(trainML['Embarked'].unique())
trainML['Embarked'] = le.transform(trainML['Embarked'])

/tmp/ipython-input-3455614950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainML['Embarked'] = le.transform(trainML['Embarked'])


# Building a Logistic Regression Classifier


In [ ]:
#create input-data and target-data
x_data = trainML[['Pclass', 'Sex', 'Age', 'Parch','Fare', 'Embarked', 'Name_len', 'Ticket_First', 'FamilyCount','title']]
y_data = trainML['Survived']

In [ ]:
#split the data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42, stratify=y_data)


In [ ]:
# Normalize the input features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# fit logistic regression

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 10)
classifier.fit(x_train, y_train)


LogisticRegression(random_state=10)

In [ ]:
# test fited logistic regression
y_pred_train = classifier.predict(x_train)
y_pred_test = classifier.predict(x_test)


In [ ]:
# evaluate the performance on train and test set

# evaluate the performance on train and test set
from sklearn.metrics import confusion_matrix, accuracy_score
print('Accuracy on train set: {:.2f}'.format(accuracy_score(y_train, y_pred_train)))
print('Accuracy on test set: {:.2f}'.format(accuracy_score(y_test, y_pred_test)))


Accuracy on train set: 0.82
Accuracy on test set: 0.79


In [ ]:
confusion_matrix(y_train, y_pred_train)

array([[341,  43],
       [ 68, 169]])

In [ ]:
# prompt: fit random forest classifier

from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(random_state=42)
classifier_rf.fit(x_train, y_train)
# predict [inference]
y_pred_train = classifier_rf.predict(x_train)
y_pred_test = classifier_rf.predict(x_test)
print('Accuracy on train set: {:.2f}'.format(accuracy_score(y_train, y_pred_train)))
print('Accuracy on test set: {:.2f}'.format(accuracy_score(y_test, y_pred_test)))



Accuracy on train set: 1.00
Accuracy on test set: 0.81
